# 🔥 Au-delà de la tolérance : cartographier les excès de vitesse extrêmes en France

L’ensemble des **12,8 millions** de prises de vitesse instantanées réalisées en 2023 par les « voitures-radars » du réseau national trace, minute-par-minute, une radioscopie sans précédent du comportement des automobilistes français.  
Accédez **directement aux données brutes** sur data.gouv.fr via : [https://static.data.gouv.fr/resources/jeux-de-donnees-des-vitesses-relevees-par-les-voitures-radars-a-conduite-externalisee/20241029-142313/opendata-vitesses-pratiquees-voitures-radars-2023-01-01-2023-12-31-.csv](https://static.data.gouv.fr/resources/jeux-de-donnees-des-vitesses-relevees-par-les-voitures-radars-a-conduite-externalisee/20241029-142313/opendata-vitesses-pratiquees-voitures-radars-2023-01-01-2023-12-31-.csv).  
Ce jeu de données regroupe pour chaque mesure : date/heure précise (format `YYYY-MM-DD hh:mm`), coordonnées GPS (lat, lon), vitesse mesurée et limite théorique du tronçon. L’objectif ici est ciblé : isoler les situations où la vitesse mesurée dépasse la vitesse autorisée **de plus de 50 km/h**, un seuil au-delà duquel l’excès est considéré comme **dangereusement critique**. En restituant la localisation exacte de ces événements, nous créons une carte de chaleur interactive pour **identifier les zones où le risque routier culminera avant qu’un drame ne survienne** – un outil directement mobilisable pour les élus, les services départementaux de sécurité, ou les associations de victimes.

## Méthodologie

L’analyse s’appuie sur DuckDB, un moteur SQL « on-the-fly » performant sur fichiers CSV volumineux.  
1. **Filtrage statique** : requête SQL (`SELECT ... WHERE mesure - limite > 50`) pour extraire les 10 000 premières observations classées par dépassement décroissant.  
2. **Parsing GPS** : conversion en champs `latitude`, `longitude` à l’aide de `split_part`.  
3. **Projection cartographique** : le centrage de la carte dynamique est automatiquement calculé sur la moyenne des points géolocalisés, puis une **HeatMap Folium** (rayon 15 m, flou 10 m, opacité min 0,6) est superposée pour rendre l’intensité de l’extrême visible d’un seul coup d’œil.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/jeux-de-donnees-des-vitesses-relevees-par-les-voitures-radars-a-conduite-externalisee/20241029-142313/opendata-vitesses-pratiquees-voitures-radars-2023-01-01-2023-12-31-.csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
    substr("date",1,10) as date_mesure,
    split_part("position",' ',1) as latitude,
    split_part("position",' ',2) as longitude,
    cast("mesure" as double) - cast("limite" as double) as depassement_kmh
FROM loaded_dataset
WHERE cast("mesure" as double) - cast("limite" as double) > 50
ORDER BY depassement_kmh DESC
LIMIT 10000 """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

Cette carte de chaleur a été réalisée avec la bibliothèque Folium, qui combine la puissance de Leaflet.js et la simplicité de Python pour produire des cartes web interactives. Elle permet de visualiser instantanément les zones où le dépassement de vitesse est le plus critique, en exploitant la densité de points GPS comme intensité de couleur.

In [3]:
import pandas as pd
import duckdb as ddb
import folium
from folium.plugins import HeatMap

df_exces = df[df['depassement_kmh'] > 50].copy()
df_exces[['latitude', 'longitude']] = df_exces[['latitude', 'longitude']].astype(float)

center_lat = df_exces['latitude'].mean()
center_lon = df_exces['longitude'].mean()

dataviz = folium.Map(location=[center_lat, center_lon], zoom_start=6)
HeatMap(df_exces[['latitude', 'longitude']], radius=15, blur=10, min_opacity=0.6).add_to(dataviz)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*